# Chapter 02: Introduction to Next-generation sequencing (NGS)
___

In this chapter, we will illustrate the properties of the NGS data and the pipeline to analyze the data. Let's have our feet wet with the sequencing data, and begin to map the reads against the genome.

## 1. Introduction

Next generation sequencing (a.k.a high-throughput sequencing), a term used to describe a set of the state-of-the-art sequencing technologies, has revolutionalize the modern investigation of molecular biology:
- **Illumina** (Solexa) Sequencing
- **Roche 454** Sequencing
- **ABI SOLiD** Sequencing
- **Ion Torrent**: Proton/PGM Sequencing

The reads can be either single-end or paired-end (mate pair).

### 1.1 Advantages of NGS

- Shorter read length and more repeats

- Much higher efficiency: massively parallel, high-throughput

- Lower cost

- Higher accuracy

### 1.2 Key Tasks in NGS Sequence Analysis

- Sequence formats and quality assessment (序列格式与序列质量评估).
- Mapping of NGS to reference sequences (比对到参考基因组序列).
- Identification and labelling of variants, such as SNPs and structural variations (发现序列中变异如SNVs和SVs).
- Sequence assembly (序列拼接).
- etc.

### 1.3 Raw Sequencing Data

- **FASTQ** : name + sequence +  associated **per-base Phred quality score**.

- Here is an example:
```
@SEQ_ID
GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT
+
!''*((((***+))%%%++)(%%%%).1***-+*''))**55CCF>>>>>>CCCCCCC65
```

#### Illumina sequence identifiers

Sequences from the Illumina software use a systematic identifier:
```
@EAS139:136:FC706VJ:2:2104:15343:197393/1:Y:18:ATCACG
```
which contains the following information:

| **Term** | **Description** |
| --- | --- |
| **EAS139** |	the unique instrument name |
| **136**	| the run id |
| **FC706VJ** |	the flowcell id |
| **2** | flowcell lane |
| **2104** | tile number within the flowcell lane |
| **15343** | 'x'-coordinate of the cluster within the tile |
| **197393** |	'y'-coordinate of the cluster within the tile |
| **1** | the member of a pair, 1 or 2 (paired-end or mate-pair reads only) |
| **Y**	 | Y if the read is filtered, N otherwise |
| **18** | 0 when none of the control bits are on, otherwise it is an even number |
| **ATCACG** | index sequence |


### 1.4 PHRED score system

- A Phred score of a base is
$$
Q_{phred} = -10 \log_{10} e
$$
where $e$ is the estimated probability of a base being an sequencing error.

- **Phred+33**: Illumina 1.8+, converted to ASCII corresponding to $Q_{phred}+33$.

### <font color="red">$\S$ Exercise 1</font>

1. Write a Python script to convert the **PHRED+33 ASCII-encoded quality string** to the corresponding estimated probability of the base being a sequencing error.

2. Given a FASTQ-formatted read below, can you compute the probability that the sequenced read is completely correct? Is this a good or bad read?
```
@IL4_315:7:105:408:43
ATTTGGCTCTCTGCTTGTTTATTATTGGTGTATNGG
+
+1,1+16;>;166>;>;;>>;>>>>>>,>>>>>+>>
```
3. Draw a histogram of the overall quality distribution in the file `data/example.fq`.

4. Use boxplot to illustrate the quality distribution for each sites across the reads in the file `example.fq`. 

In [1]:
def Phred33ToQ(qual):
    """convert the qual character into the quality value."""
    return ord(qual)

def QToErrorRate(Q):
    """convert the quality value to the error probability."""
    pass

def ReadFastqFile(fname):
    """Read a fastq file and return a list of sequences and corresponding qualities"""
    seqs = []
    quals = []
    fh = open(fname)
    while True:
        fh.readline()
        seq = fh.readline().rstrip()
        fh.readline()
        qual = fh.readline().rstrip()
        if len(seq)==0:
            break
        seqs.append(seq)
        quals.append(qual)
    return seqs, quals

def QToHistogram(quals):
    """Convert a list of qualities into a histogram."""
    pass

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
seqs, quals = ReadFastqFile("data/example.fq")
quals = Phred33ToQ(quals)
hist = QToHistogram(quals)
plt.bar(range(len(hist)), hist)
plt.boxplot(hist)) 

SyntaxError: invalid syntax (<ipython-input-3-8bf80f31ba80>, line 5)

[Here](demo/boxplot.py) is a demo for drawing a boxplot.

## 1. Mapping: Align the reads to the reference genome

As a popular efficient algorithm in text matching and full-text query, now BWT has become one of the efficient tools in mapping reads to the reference genome.

### 1.1 Burrows-Wheeler transform (BWT)

For a input string, how can we build the sorted Burrows-Wheeler array (BWA) and conduct the Burrows-Wheeler transform?

It's very simple by
- first **compute all the possible rotations of the string**, and 
- then **sort them lexically**. 

![](../images/bwt1.gif)

Therefore, $BWA(T)$ is the lexical sorting of `T`, right?

In [ ]:
def bwa(t):
    t = t + "$"
    bwa = []
    s0 = []
    lt = len(t)
    for i in range(lt):
        bwa.append(t[i:] + t[:i])
        s0.append((t[i:] + t[:i], i))
    return sorted(bwa), sorted(s0)

sa = {}
rank = {}
array = bwa("ACAACG")[1]
for i, s in enumerate(array):
    print "{}\t{}\t{}".format(i, s[0], s[1])
    sa[i] = s[1]
    rank[s[1]] = i

And here are the suffixes:

In [ ]:
suffix_sorted = [array[i][0][:array[i][0].index('$')+1] for i in xrange(len(array))]
for st in suffix_sorted:
    print st

Here `sa` is the corresponding suffix array and `rank` is its reverse.

In [ ]:
for i in sa:
    print i, sa[i]

In [ ]:
for _ in rank:
    print _, rank[_]

And then write down the last column as the `BWT(T)`:

In [ ]:
def bwt(t):
    bw = bwa(t)[0]
    lt = len(t)
    bwt = [seq[lt] for seq in bw]
    
    return ''.join(bwt)

print bwt("ACAACG")

### 1.2 Reconstruct the sequence from the BWT string

![](../images/bwt2.gif)
![](../images/bwt3.gif)

In [ ]:
bwt_string = 'GC$AAAC'

def occ(ch):
    """Compute number of characters that are literally smaller than ch"""
    return sum([c<ch for c in bwt_string])

def count(ch, idx):
    """Count the occurrences of ch in bwt before idx"""
    return sum([c==ch for c in bwt_string[:idx]])

def LF(ch,  idx):
    """Last-to-First conversion for ch at idx in bwt"""
    return occ(ch) + count(ch, idx)

def reconstruct(bwt_str):
    """Reconstruct the genome from the bwt"""
    i = 0
    t = ''
    while not bwt_str[i] == '$':
        t = bwt_str[i] + t
        print i
        i = LF(bwt_str[i],  i)
        
    return t

seq = reconstruct(bwt_string)
print seq

![](../images/bwt4.gif)

## 2. Full-text Minute-size index (FM Index)

How do we convert a genome into an alternate representation that permits rapid matching of millions of sequence reads? FM-index and BWT, in my opinion, stands for the same thing, is used for creating index for a long text.

## 3. Read Alignment

How can we use an FM index and BWT to rapidly align reads to a refernce genome?

### 3.1 Exact matching with FM Index

For a query sequence, we can use an iterative method to align:

In [ ]:
def bwtQuery(query, genome):
    index = bwt( genome)
    top = 0
    bot = len(index)
    for ch in query[::-1]:
        top = LF(ch, top)
        bot = LF(ch, bot)
    return top, bot

genome = "ACAACG"
query = "AAC"
hits = bwtQuery(query, genome)
print "Finally hit at {}".format(hits)
print "Find {} at position {} of genome {}".format(query, sa[hits[0]], genome)

### <font color="red">Explanation of the algorithm</font>

In each iteration, **top** and **bot** delimit the range of rows beginning with progressively longer suffixes of the `query` (i.e., `top` and `bot`defines the ranges of rows of Burrows-Wheeler Array (BWA) that exactly match the current suffix of the `query` reads), and suffix of the `query` extends a base longer in each iteration.  So if range becomes empty (that is, `top = bot`), the `query suffix` (and therefore the `query` itself) does not occur in the `genome`.

So we can see that the time complexity of the matching is $\mathcal{O}(m)$ where $m$ is the length of the reads. That is, the algorithm is linear complexity, right?

So once we've built the index of the genome, we can use it again and again. We don't need to rebuild the index, right? You don't have to pay for the time to build this index.

### 3.2 Find the exact hits in the genome

Now we know that our query has a hit in the genome,  but the question is, 

> ### where is the hit in the genome?

The only fact that it matched row 1 of the Burrows-Wheeler array (BWA) does me absolutely no good at all. 

Now we have BWT, OCC, and Count, we can compute the LF (Last-to-First, mapping last column to first column) function. How can we do with the information to locate the hit in the genome?

#### 3.2.1 Naive solution 1

We can use the "**walk-left**" to walk back to the beginning of the genome (that is, the row ending with `$` sign) and the number of steps to take is exactly the **offset of hit** in the genome.

In [ ]:
i = hits[0]
offset = 0
while not bwt_string[i]=='$':
    ch = bwt_string[i]
    i = LF(ch, i)
    offset += 1
print offset

### <font color="red">$\S$ Exercise 2</font>

If we have multiple hits, that is, there are more than one row betwen `top` and `bottom`, say, 2 and 5. What should we do to obtain all the hits (offsets in the genome)?

This algorithm, is also linear, but in the length of the genome, $\mathcal{O}(n)$. Due to the large size of the genome, this could be too slow.

But any other ideas can help?

### 3.2.2 Naive solution 2: Rows to Reference Positions

We can keep the whole suffix array in the memory. Thus it is a **lookup** in the array to find the reference position. That is, when we build the index, we need to store the offset for each row of the suffix array:
> ### However, for human genome, the suffix array is $\sim 12 GB$, which is too big.

### 3.2.3 Hybrid solution


![](../images/bwt5.gif)
Store not all, but sample of suffix array, and then "walk left" to the next sampled (a.k.a "marked") row to the left. This approach is proposed by Ferragina and Manzini in 2008. This, can cut down the storage demand.

混合方法是上述两种策略的折中，只保存部分后缀数组，然后采用上述第一种“向左游走”策略，直到找到下一个“被标记”的后缀数组行为止。将“向左游走”的步数加上该行的“偏移量”，得到的结果就是输入检索序列在基因组上的偏移位置。

Thus the final hit offset is the sum of the "walk-left" steps and the "marked" row offset of the sampled suffix array. 

One of the ngs short read aligner, Bowtie, "marks" every 32nd  row by default (but note that this can be modifiable).

### 3.2.4 Improving the LF() function

Now the alignment of short reads against the genome is linear. However, in the inner iteration, the `LF()` routine is still very time-consuming. `LF(ch, idx)`tries to determine the **rank** of *ch* in row $idx$, which means that we need to count the occurrences of `ch` in all previous rows. This is very expensive, although the rank is a very simple metric.

![](../images/bwt6.gif)

Instead of doing that, we can build a data structure that pre-calculate the cumulative counts for A/C/T/G up to **periodic checkpoints** in BWT. So when you need to compute the counts at an arbitrary point, you can go to the nearest checkpoint and make an adjustment by counting the number of characters betwen the index and the checkpoint. A very straight-forward approach, all right.

![](../images/bwt7.gif)

So now we know that the full index consists of 
- the **Burrows-Wheeler transform (BWT)** with the same size as $T$, 
- the **sampling of the suffix array** with around $50\%$ of $T$
- the **checkpoint** for the `LF()` function to make a constant time ($~15\%$ of $T$)

That's what is inside of an FM-Index. Therefore the size of full FM-index is about $1.65x$ the size of $T$.

In Bowtie, 
- Two bits for the encoding of each base and no compression
- 16 bytes checkpoint for every 448 characters
- Use default suffix sampling rate

We can see that it is very small, compared to things like **suffix trees ($>45x$)**, **suffix arrays ($>15x$) **, or even **other kinds of hash indexes ($>15x$) for looking for seeds**.

So it's a very compact index, and thus it is very efficient. It is a very wonderful data structure, except that we have not dealt with mismatches yet, right?

### References

#### Oligomer counting
- Healy J *et al*: Annotating large genomes with exact word matches. *Genome Res* 2003, 13(10):2306-2315.
   
#### Whole-genome alignment
- Li H *et al*: Fast and accurate short read alignment with Burrows-Wheeler transform. *Bioinformatics* 2009, 25(14): 1754-1760 (bwa aligner)
    
#### Smith-Waterman alignment to large reference
- Lam TW *et al*: Compressed indexing and local alignment of DNA 

### <font color="red">$\S$ Exercise 3</font>

Please organize the above python scripts into a object-oriented fashion.

```python
class BWT():
    """add your code here"""
    def __init__(self, bwt):
        pass
    
    def build_bwt(genome):
        pass
        
    ...

```

And also run some examples to verify your codes.

### Appendix: How to deal with mismatches?

Once we are trapped by `top==bot`, which means that the query does not exactly occur in the genome. But we can change the current character until it can go ahead. This is called the **backtracking**.

This method can also be applied when we want to take into accout the sequencing quality of the reads. We know that for a sequenced read, it is often of the poor quality in the 3' end. But the above alignment are starting from the right-most bases, so this often results in false hits. So for the bases with poor quality, we need to change the state since we are not confident of the sequenced results.

Additionaly, to deal with the 3'-end bias of sequencing error, we can use the "mirror" index of the genome and also reverse the query string in order to get a better result.

All these wonderful technique, can overcome some of the limitations of backtracking. Unfortunately, this will of course increase the computational burden. This requires you to make a balance between these two metrics - accuracy and complexity.

> When creating BWT, Heng Li's **bwa** does not sort the entire Burrow-Wheeler array. Instead, he use an insertion-based method to achieve a better efficiency.

## 4. The Alignment Pipeline

The typical pipeline for alignment contains the following 5 steps:

### 4.1 Obtain the reads and do some quality checking (QC)

This is the data propressing step. In this step, besides obtaining the reads data, you need to trim the adapters and the poor-quality bases/reads from the data.

The typical tools for this step usually include:
- **[SRA Toolkit](https://github.com/ncbi/sra-tools)**'s `fastq-dump` for extracting `*.fastq` out of the `*.sra` file;
- **[FastQC](www.bioinformatics.babraham.ac.uk/projects/fastqc)** for quality checking of the `fastq` files;
- **[Picard Tools](https://broadinstitute.github.io/picard)** for marking the duplicates;
- **[Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic)** for low-quality trimming of the reads;
- **[FastX Toolkit](http://hannonlab.cshl.edu/fastx_toolkit)** for processing the reads.

### 4.2 Conduct the alignment (mapping)

To accomplish the task, you need to finish the following jobs:
- Index the genome sequence;
- Align the reads to the genome;
- Sort the alignments;
- Merge the alignments;
- Recalibrate the alignments;
- Merge the libraries;
- Index the final alignments. 

The typical tools for this step include:
- The **alignment tools** (e.g. [bwa](http://bio-bwa.sourceforge.net/); [bowtie](bowtie-bio.sourceforge.net/); [SOAP](http://soap.genomics.org.cn/)) 
- **[samtools](http://samtools.sourceforge.net/)**: providing various utilities for manipulating alignments in the [SAM](http://samtools.sourceforge.net/SAM1.pdf) format, including sorting, merging, indexing and generating alignments in a per-position format. 
- **[GATK](https://software.broadinstitute.org/gatk/)**: offering a wide variety of tools with a primary focus on variant discovery and genotyping.

### 4.3 `bwa`

- `bwa index  [-a bwtsw|div|is] [-c] <in.fasta>`
    * **Burrows-Wheeler Transform** construction algorithm with "bwtsw" for vertebrate-size genomes and "is" for smaller genomes.
    
- `bwa aln [options] <prefix> in.fq`
    * aligns each single-ended fastq file individually;
    * `<prefix>` is the name of the reference file spcified by `bwa index`;
    * `[options]` controls alignment parameters, scoring matrix, seed length, etc.

- `bwa sampe <prefix> <in1.sai> <in2.sai> <in1.fq> <in2.fq>`
    * generates pairwise alignment from both `<in1.sai>` and `<in2.sai>` files produced by `bwa aln`;
    * for paired-ended reads stored in `<in1.fq>` and `<in2.fq>`;
    * produces `SAM` outputs.

- `bwa bwasw <prefix> <query.fa>`
    * aligns long reads in `<query.fa>` to the reference named `<prefix>`;
    * produces `SAM`-formatted outputs.

#### <font color="red">`bwa` usage notes</font>
- `bwa` finds matches up to a preset **edit distance** (by default, for a 100 bases allowing 5 edits)
- `bwa` use `-q` to set clipping for **quality threshold**, e.g. 20.
- ** Ambiguous (non-ACGT)** bases are treated as **mismatch**.
- **Parallelization** for speed:
    * split data into 1Gb blocks;
    * each 1Gb blocks takes around 8 hours.
- Check for truncated BAM files (e.g. using `samtools flagstats`)

### 4.4 How to further improve the alignments?

To improve the alignments generated by `bwa`, you need to do the following steps:
- **Remove the duplicates in the library**, which can utilize such tools like **samtools** and **Picard**.
- **Realignment around the indels** and **Base quality calibration** using **GATK**. 


### Library duplicate removal

PCR amplification is essential in the sequencing of **low-input** DNA samples. But unfortunately, this will result in duplicate DNA fragments, which can further lead to false SNP calls.

To remove the duplicates, you can first **identify the read pairs whose outer ends map to the same position** on the genome and then **remove all but one copy**. This can be done by the two similar commands:
- `samtools rmdup`
- `Picard/GATK MarkDuplicates`

### Realignment

Short indels in the sample relative to the reference can pose difficulties for alignment task since
- indels near the ends of the reads are often not aligned correctly;
- the aligners prefer naturally to introducing SNPs rather than indels.

But how to do realignment?  Usually, the realignment algorithm works by
- Input set of known indels (e.g., previously publised indel sites, dbSNPs, 1K-Genomes, or estimated directly from the alignments) and a BAM file;
- At each site, model the indels and the reference haplotype, and select best fit with data;
- Generate and modify new BAM files where the indels have been introduced through the realignment;
- This can be implemented in `GATK indelRealign` task).

### 4.5 Additional alignment issues

We'd better separate chromosome BAM files in order to parallelly process them easier.

Moreover, for unmapped reads, we'd better conduct a further realignment to avoid missing due to reference incompatibillity or incompleteness issues.

## 5. BAM/SAM

SAM (Sequence Alignment/Map format) is an unified format for storing NGS alignment against the reference genome.

Here are the some features for SAM/BAM format:
- SAM file stores the alignment information from most alignment tools;
- SAM supports almost all the sequencing technologies;
- SAM supports indexing for quick retrieval;
- Reads in the SAM file can be classified into logical groups (e.g., lanes, libraries, or individuals)

The BAM (binary alignment/map format) is the binary equivalence of SAM.

### 5.1 SAM format

A typical SAM file contains two sections - the HEADER section and the ALIGNMENT section.

The ALIGNMENT section stores the alignment information for the reads with each one line. Each line contains 11 mandatory fields and several optional fields (formatted as TAG TYPE VALUE). Here is the table for the list of mandatory fields:

| Col | Field | Type | Description |
| --- | --- | --- | --- |
| 1 |  QNAME | str | query name of the read or the read pair |
| 2 | FLAG | int | bitwise flags (pairing, mapped, mate mapped, etc.) |
| 3 | RNAME | str | reference sequence name |
| 4 | POS | int | 1-based leftmost position of clipped alignment |
| 5 | MAPQ | int | PHRED-scale mapping quality |
| 6 | CIGAR | str | extended CIGAR string for alignment details |
| 7 | RNEXT | str | mate reference name ('=' for the same) |
| 8 | PNEXT | int | position of mate/next segment |
| 9 | TLEN | int | observed template length |
| 10 | SEQ | str | segment sequence |
| 11 | QUAL | str | ASCII of PHRED-scale base quality | 

For further about SAM/BAM format, please refer to https://samtools.github.io/hts-specs/SAMv1.pdf. 

For FLAG information, you can refer to http://picard.sourceforge.net/explain-flags.html.

### 5.2 SAM/BAM file processing tools
- **`samtools`**: C program and library
    * http://samtools.sourceforge.net
    * `samtools view`: SAM-BAM conversion
    * `samtools sort`: sort the SAM records according to the positions
    * `samtools index`:  create the index for SAM records
    * `samtools merge`: merge  multiple BAM files
    * `samtools flagstats`: summarizes the mapping flags
    
- **`Picard`**: Java program suite
    * http://picard.sourceforge.net
    * `MarkDuplicates`, `CollectAlignmentSummaryMetrics`, `CreateSequeceDictionary`, `SamToFastq`, `MeanQualityByCycle`

- **`Pysam`**: Python interface to samtools
    * http://code.google.com/p/pysam

## 6. Variant Calling

The vaiant calling step will generate a great deal of calling
- Single nucleotide variants (SNVs) with genotype information (homozygous or heterozygous)
- indels and
- structural variants (SVs)

This is a list of variant calling tools:
- [samtools](http://samtools.sourceforge.net), [bcftools](http://bcftools.sourceforge.net)
- [GATK](http://gatk.broadinstitute.org), [SOAPsnp](http://soap.genomics.org.cn/soapsnp.html), [Dindel](https://sites.google.com/site/keesalbers/soft/dindel)
- [SVMerge](http://svmerge.sourceforge.net)

These calling tools will generate a resulting file in [VCF](https://samtools.github.io/hts-specs/VCFv4.2.pdf) or [pileup](samtools.sourceforge.net/pileup.shtml).

The calling and the filtering protocols will take advantages of the depth, quality, strand bias as well as the multiple sample information to accomplish the calling task.

Compared to SNPs, the indels and the structural variants are much harder to call.
